<a href="https://colab.research.google.com/github/XiaoqiW09/Sigkernel/blob/main/sigkernel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install git+https://github.com/crispitagorico/sigkernel.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/crispitagorico/sigkernel.git to /tmp/pip-req-build-7qvt8_te
  Running command git clone -q https://github.com/crispitagorico/sigkernel.git /tmp/pip-req-build-7qvt8_te


In [3]:
pip install signatory

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Created wheel for signatory: filename=signatory-1.2.6.1.9.0-cp37-cp37m-linux_x86_64.whl size=7554077 sha256=e34b871a89cc5eb1d79cdc0b7fb52008900af90c95f43facf98919df17e0c04b
  Stored in directory: /root/.cache/pip/wheels/12/ff/e5/ffe613433c810f7f82da6e0c55abd15f4cc04960f8137db53b
Successfully built signatory


In [4]:
pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 19.8 MB/s 


In [5]:
pip install iisignature

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 38.4 MB/s 
  Created wheel for iisignature: filename=iisignature-0.24-cp37-cp37m-linux_x86_64.whl size=2528292 sha256=640205f1b9bbb86dcdeed888f9e7b2fa5415de7ee6baf24b91fc9d81cc2a288a
  Stored in directory: /root/.cache/pip/wheels/d6/99/a3/b2b3f95df7c69f664e92b4ec58b8da922cb95bc394700cbc0f
Successfully built iisignature


In [6]:
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
import signatory
import torch
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sigkernel
import torch 
import math
import pickle
from time import sleep
import iisignature as sig

from sklearn.preprocessing import LabelEncoder,StandardScaler
from tslearn.datasets import UCR_UEA_datasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance,TimeSeriesScalerMinMax
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from tslearn.svm import TimeSeriesSVC
from sklearn.svm import SVC
from tslearn.datasets import UCR_UEA_datasets
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tslearn.svm import TimeSeriesSVC
from tslearn.preprocessing import TimeSeriesScalerMinMax
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
import joblib
import h5py
import mne

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
data_list = ['alcoholic_1', 'alcoholic_12', 'alcoholic_21']

In [9]:
def write_alcoholic(dataset='alcoholic_1'):
    # S1: S1 obj - a single object shown;
    s1 = 0
    # S12: S2 nomatch - object 2 shown in a non-matching condition (S1 differed from S2)
    s12 = 0
    # S21: S2 match - object 2 shown in a matching condition (S1 was identical to S2),
    s21 = 0
    # initialise numpy arrays to fill with time series
    s1_X_train_unscaled = np.zeros((160, 256, 64))
    s1_y_train = np.zeros(160)
    s21_X_train_unscaled = np.zeros((159, 256, 64))
    s21_y_train = np.zeros(159)
    s12_X_train_unscaled = np.zeros((149, 256, 64))
    s12_y_train = np.zeros(149)

    # assign numerical values to the classes
    classifier = {'a': 1, 'c': 0}

    # run through every file in the train directory and import it, using information from the
    # matching condition column to determine which experiment was being conducted.
    # using this, put the data into the corresponding numpy array
    filenames_list = os.listdir('/content/drive/My Drive/alcohol train data')

    for file_name in tqdm(filenames_list):
        temp_df = pd.read_csv('/content/drive/My Drive/alcohol train data/' + file_name)
        if temp_df["matching condition"][0] == "S1 obj":
            s1_X_train_unscaled[s1] = np.transpose(np.array(temp_df["sensor value"]).reshape([64, 256]))
            s1_y_train[s1] = classifier[temp_df['subject identifier'][0]]
            s1 += 1
        if temp_df["matching condition"][0] == "S2 match":
            s21_X_train_unscaled[s21] = np.transpose(np.array(temp_df["sensor value"]).reshape([64, 256]))
            s21_y_train[s21] = classifier[temp_df['subject identifier'][0]]
            s21 += 1
        if temp_df["matching condition"][0] == "S2 nomatch,":
            s12_X_train_unscaled[s12] = np.transpose(np.array(temp_df["sensor value"]).reshape([64, 256]))
            s12_y_train[s12] = classifier[temp_df['subject identifier'][0]]
            s12 += 1

    # t1: S1 obj - a single object shown;
    t1 = 0
    # t12: S2 nomatch - object 2 shown in a non-matching condition (S1 differed from S2)
    t12 = 0
    # t21: S2 match - object 2 shown in a matching condition (S1 was identical to S2),
    t21 = 0
    t1_X_test_unscaled = np.zeros((160, 256, 64))
    t1_y_test = np.zeros(160)
    t21_X_test_unscaled = np.zeros((160, 256, 64))
    t21_y_test = np.zeros(160)
    t12_X_test_unscaled = np.zeros((160, 256, 64))
    t12_y_test = np.zeros(160)

    # same as above but for test data
    classifier = {'a': 1, 'c': 0}

    # list of filenames in the directory
    filenames_list = os.listdir('/content/drive/My Drive/UROP alcohol test data')

    for file_name in tqdm(filenames_list):
        if file_name == "Test":
            pass
        else:
            temp_df = pd.read_csv('/content/drive/My Drive/UROP alcohol test data/' + file_name)
            if temp_df["matching condition"][0] == "S1 obj":
                t1_X_test_unscaled[t1] = np.transpose(np.array(temp_df["sensor value"]).reshape([64, 256]))
                t1_y_test[t1] = classifier[temp_df['subject identifier'][0]]
                t1 += 1
            if temp_df["matching condition"][0] == "S2 match":
                t21_X_test_unscaled[t21] = np.transpose(np.array(temp_df["sensor value"]).reshape([64, 256]))
                t21_y_test[t21] = classifier[temp_df['subject identifier'][0]]
                t21 += 1
            if temp_df["matching condition"][0] == "S2 nomatch,":
                t12_X_test_unscaled[t12] = np.transpose(np.array(temp_df["sensor value"]).reshape([64, 256]))
                t12_y_test[t12] = classifier[temp_df['subject identifier'][0]]
                t12 += 1

    if dataset == 'alcoholic_1':
        X_train = s1_X_train_unscaled
        y_train = s1_y_train
        X_test = t1_X_test_unscaled
        y_test = t1_y_test
    elif dataset == 'alcoholic_12':
        X_train = s12_X_train_unscaled
        y_train = s12_y_train
        X_test = t12_X_test_unscaled
        y_test = t12_y_test
    elif dataset == 'alcoholic_21':
        X_train = s21_X_train_unscaled
        y_train = s21_y_train
        X_test = t21_X_test_unscaled
        y_test = t21_y_test
    else:
        X_train = None
        y_train = None
        X_test = None
        y_test = None

    # f = h5py.File("/content/drive/My Drive/data/data.h5", 'a')
    # grp = f.create_group(f"alcoholic_{subset}")
    # grp.create_dataset("X_train", data=X_train, compression="gzip", compression_opts=7)
    # grp.create_dataset("y_train", data=y_train, compression="gzip", compression_opts=7)
    # grp.create_dataset("X_test", data=X_test, compression="gzip", compression_opts=7)
    # grp.create_dataset("y_test", data=y_test, compression="gzip", compression_opts=7)

    return X_train, y_train, X_test, y_test

In [ ]:
def move_to_hdf():
    for i in tqdm(os.listdir("/content/drive/My Drive/data")):
        if i.endswith("npz"):
            npz = np.load(f"data/{i}", allow_pickle=True)
            X_train = npz['arr_0']
            y_train = npz['arr_1']
            X_test = npz['arr_2']
            y_test = npz['arr_3']

            print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep="\n")

            f = h5py.File("/content/drive/My Drive/data/data.h5", 'a')
            grp = f.create_group(i[:-4])
            grp.create_dataset("X_train", data=X_train, compression="gzip", compression_opts=7)
            grp.create_dataset("y_train", data=y_train, compression="gzip", compression_opts=7)
            grp.create_dataset("X_test", data=X_test, compression="gzip", compression_opts=7)
            grp.create_dataset("y_test", data=y_test, compression="gzip", compression_opts=7)

In [ ]:
def read_data(dataset="alcoholic_1"):
    data = h5py.File("/content/drive/My Drive/data/data.h5", 'r')
    X_train = data[f"{dataset}/X_train"][:]
    y_train = data[f"{dataset}/y_train"][:]
    X_test = data[f"{dataset}/X_test"][:]
    y_test = data[f"{dataset}/y_test"][:]

    return X_train, y_train, X_test, y_test

In [ ]:
### ml methods ###

In [ ]:
def ml_method_setup(method, X_train, sig_train, y_train, file_name, reduced=False):
    if method == 'ts_knn':
        
        
            if reduced:
                clf = GridSearchCV(
                    Pipeline([
                        ('knn', KNeighborsTimeSeriesClassifier())
                    ]),
                    {'knn__n_neighbors': range(3, 30, 6), 'knn__weights': ['uniform', 'distance']},
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0),
                    n_jobs=-1,
                    verbose=10
                )
            else:
                clf = GridSearchCV(
                    Pipeline([
                        ('knn', KNeighborsTimeSeriesClassifier())
                    ]),
                    {'knn__n_neighbors': range(3, 30, 2), 'knn__weights': ['uniform', 'distance']},
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0),
                    n_jobs=-1,
                    verbose=10
                )
            clf.fit(X_train, y_train)
            joblib.dump(clf.best_estimator_, f'/content/drive/My Drive/models/{file_name}.pkl')

    elif method == 'ts_svc':
      
        
            if reduced:
                clf = GridSearchCV(
                    Pipeline([
                        ('svc', TimeSeriesSVC(random_state=0, probability=True))
                    ]),
                    {'svc__kernel': ['rbf', 'poly'], 'svc__shrinking': [True, False],
                     'svc__C': [0.1, 1, 10]},
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0),
                    n_jobs=-1,
                    verbose=10
                )
            else:
                clf = GridSearchCV(
                    Pipeline([
                        ('svc', TimeSeriesSVC(random_state=0, probability=True))
                    ]),
                    {'svc__kernel': ['gak', 'rbf', 'poly'], 'svc__shrinking': [True, False],
                     'svc__C': [0.1, 0.2, 0.5, 1, 2, 5, 10]},
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0),
                    n_jobs=-1,
                    verbose=10
                )
            clf.fit(X_train, y_train)
            joblib.dump(clf.best_estimator_, f'/content/drive/My Drive/models/{file_name}.pkl')

    elif method == 'lr':
        
       
            lr = LogisticRegression(random_state=0)
            parameters = {'C': [0.1, 0.2, 0.5, 1, 2, 5, 10],
                          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
            clf = GridSearchCV(lr, parameters, n_jobs=-1, verbose=10)
            clf.fit(sig_train, y_train)
            joblib.dump(clf.best_estimator_, f'models/{file_name}.pkl')

    elif method == 'svc':
        
            svc = SVC(random_state=0, probability=True)
            if reduced:
                parameters = {'kernel': ['rbf', 'poly'], 'shrinking': [True, False],
                              'C': [0.1, 1, 10]}
            else:
                parameters = {'kernel': ['rbf', 'poly'], 'shrinking': [True, False],
                              'C': [0.1, 0.2, 0.5, 1, 2, 5, 10]}
            clf = GridSearchCV(svc, parameters, n_jobs=-1, verbose=10)
            clf.fit(sig_train, y_train)
            joblib.dump(clf.best_estimator_, f'models/{file_name}.pkl')

    elif method == 'knn':
        
            knn = KNeighborsClassifier()
            parameters = {'n_neighbors': range(3, 30, 2), 'weights': ['uniform', 'distance']}
            clf = GridSearchCV(knn, parameters, n_jobs=-1, verbose=10)
            clf.fit(sig_train, y_train)
            joblib.dump(clf.best_estimator_, f'/content/drive/My Drive/models/{file_name}.pkl')

    elif method == 'ada':
        
            ada = AdaBoostClassifier(random_state=0)
            if reduced:
                parameters = {'n_estimators': [50, 100], 'learning_rate': [0.5, 1, 2]}
            else:
                parameters = {'n_estimators': [50, 100, 150], 'learning_rate': [0.1, 0.5, 1, 2]}
            clf = GridSearchCV(ada, parameters, n_jobs=-1, verbose=10)
            clf.fit(sig_train, y_train)
            joblib.dump(clf.best_estimator_, f'/content/drive/My Drive/models/{file_name}.pkl')

    elif method == 'rf':
        
            rf = RandomForestClassifier(random_state=0)
            if reduced:
                parameters = {'min_weight_fraction_leaf': [0.1, 0.5],
                              'bootstrap': [True, False],
                              'max_depth': (2, 5),
                              'max_leaf_nodes': (2, 5),
                              'n_estimators': (100, 200)}
            else:
                parameters = {'min_weight_fraction_leaf': [0.01, 0.1, 0.5],
                              'bootstrap': [True, False],
                              'max_depth': (2, 5, 10),
                              'max_leaf_nodes': (2, 5, 10),
                              'n_estimators': (100, 200, 300)}
            clf = GridSearchCV(rf, parameters, n_jobs=-1, verbose=10)
            clf.fit(sig_train, y_train)
            joblib.dump(clf.best_estimator_, f'/content/drive/My Drive/models/{file_name}.pkl')

    else:
        clf = None

    return clf

In [ ]:
def time_augment(arr):
    return np.vstack((arr.T, np.linspace(0, 1, num=arr.shape[0]))).T

In [ ]:

def shuffle_dataset(X_train, y_train, X_test, y_test, train_size = 0.8):
  X_total = np.concatenate((X_train,X_test),axis=0)
  y_total = np.concatenate((y_train,y_test),axis=0)
  X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, train_size = train_size)
  return X_train, X_test, y_train, y_test

In [ ]:
def sigkernel_train(X_train, y_train, X_test, y_test):
    svc_parameters = {'C': np.logspace(0, 4, 5), 'gamma': list(np.logspace(-4, 4, 9)) + ['auto']}
    _sigmas = [1e-2, 2.5e-2, 5e-2, 7.5e-2, 1e-1, 1.5e-1, 2e-1, 2.5e-1, 3e-1, 3.5e-1, 4e-1, 4.5e-1, 5e-1, 5.5e-1, 6e-1, 6.5e-1, 7e-1, 7.5e-1, 8e-1, 8.5e-1, 9e-1, 9.5e-1, 1.]
    _scales = [5e-2, 1e-1, 5e-1, 1e0]
    best_scores = 0
      
    best_scores = 0
    if X_train.shape[1] <= 200 and X_train.shape[2] <= 8: 
        transforms = tqdm([(True,True), (False,True), (True,False), (False,False)], position=1, leave=False)
    else: # do not try lead-lag as dimension is already high
        transforms = tqdm([(True,False), (False,False)], position=1, leave=False)

    for (at,ll) in transforms:
        transforms.set_description(f"add-time: {at}, lead-lag: {ll}")
        X_train_tl= X_train / X_train.max()


        # path-transform
        X_train_tl = sigkernel.transform(X_train_tl, at=at, ll=ll, scale=.1)

        X_train_tl = torch.tensor(X_train_tl, dtype=torch.float64, device='cuda')
                    
        # grid search over sigmas
  
        for sigma in _sigmas:

            # define static kernel
            static_kernel = sigkernel.RBFKernel(sigma=sigma)

            # initialize corresponding signature PDE kernel
            signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order=0)

            # compute Gram matrix on train data
            G_train = signature_kernel.compute_Gram(X_train_tl, X_train_tl, sym=True, max_batch=40).cpu().numpy()

            # SVC sklearn estimator
            svc = SVC(kernel='precomputed', decision_function_shape='ovo',probability=True)
            svc_model = GridSearchCV(estimator=svc, param_grid=svc_parameters, cv=5, n_jobs=-1)
            svc_model.fit(G_train, y_train)
                        
            # empty memory
            del G_train
            #torch.cuda.empty_cache()
            
            # store results
            if svc_model.best_score_ > best_scores:
                best_scores = svc_model.best_score_
                trained_models = (at, ll, sigma, svc_model)

            sleep(0.5)
    return best_scores,trained_models

In [ ]:
def sigkernel_test(X_train, y_train, X_test, y_test, trained_models): 


    X_train_tl= X_train / X_train.max()
    X_test_tl= X_test / X_test.max()

    at, ll, sigma, estimator = trained_models


    X_train_tl = torch.tensor(X_train_tl, dtype=torch.float64, device='cuda')
    X_test_tl = torch.tensor(X_test_tl, dtype=torch.float64, device='cuda')

    # define static kernel
    static_kernel = sigkernel.RBFKernel(sigma=sigma)
                        
    # initialize corresponding signature PDE kernel
    signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order=0)
                            
    # compute Gram matrix on test data
    G_test = signature_kernel.compute_Gram(X_test_tl, X_train_tl, sym=False,max_batch=60).cpu().numpy()

    y_pred_proba = estimator.predict_proba(G_test)[:, 1]
    y_pred = estimator.predict(G_test)
                        
    # record score
    accuracy = accuracy_score(y_test, y_pred)
    train_score = estimator.best_score_
    test_score = estimator.score(G_test, y_test)
    return accuracy,test_score,train_score

    # empty memory
    del G_test
    #torch.cuda.empty_cache()

    sleep(0.5)

In [ ]:
def auto_ml(X_train, y_train, X_test, y_test, method, sig_level, dataset, ts_scale=True, standard_scale=True,
            time_aug=False, train_size = 0.8):
    accuracy = []
    accuracy_test = []
    accuracy_train = []
    

    method_dict = {"rf": "Random Forests", "ada": "AdaBoost", "knn": "K Nearest Neighbours",
                   "svc": "Support Vector Machines", "lr": "Logistic Regression",
                   "ts_svc": "Time Series Support Vector Machines",
                   "ts_knn": "Time Series K Nearest Neighbours"}


    for i in range(5):
        X_train, X_test, y_train, y_test = shuffle_dataset(X_train, y_train, X_test, y_test, train_size)
        # initialise scalers
        ts_scaler = TimeSeriesScalerMinMax()
        scaler = StandardScaler()
        if dataset.startswith("mi"):
            X_train = np.nan_to_num(X_train[:1000])
            X_test = np.nan_to_num(X_test[:250])
            y_train = np.nan_to_num(y_train[:1000])
            y_test = np.nan_to_num(y_test[:250])

        if time_aug:
            X_train = np.array(list(map(time_augment, X_train)))
            X_test = np.array(list(map(time_augment, X_test)))

        if ts_scale:
            X_train = ts_scaler.fit_transform(X_train)
            X_test = ts_scaler.fit_transform(X_test)

        if torch.cuda.is_available():
            X_train_torch = torch.from_numpy(X_train).cuda()
            X_test_torch = torch.from_numpy(X_test).cuda()

            sig_train_unscaled = signatory.signature(X_train_torch, sig_level)
            sig_train_unscaled = sig_train_unscaled.cpu().numpy()
            sig_test_unscaled = signatory.signature(X_test_torch, sig_level)
            sig_test_unscaled = sig_test_unscaled.cpu().numpy()

        else:
            X_train_torch = torch.from_numpy(X_train)
            X_test_torch = torch.from_numpy(X_test)

            sig_train_unscaled = signatory.signature(X_train_torch, sig_level)
            sig_train_unscaled = sig_train_unscaled.numpy()
            sig_test_unscaled = signatory.signature(X_test_torch, sig_level)
            sig_test_unscaled = sig_test_unscaled.numpy()

        if standard_scale:
            sig_train = scaler.fit_transform(sig_train_unscaled)
            sig_test = scaler.fit_transform(sig_test_unscaled)
        else:
            sig_train = sig_train_unscaled
            sig_test = sig_test_unscaled

        # file name formatting
        file_name = f"{dataset}_{method}_{i}"
        if not method.startswith("ts"):
            file_name += f"_sig{sig_level}"
        if ts_scale:
            file_name += "_ts_scale"
        if standard_scale:
            file_name += "_standard_scale"
        if time_aug:
            file_name += "_time_aug"
    
        if dataset.startswith("mi"):
            clf = ml_method_setup(method, X_train, sig_train, y_train, file_name, reduced=True)
        else:
            clf = ml_method_setup(method, X_train, sig_train, y_train, file_name)
        print(i)

        # fit to data
        if method.startswith("ts"):
            y_pred_proba = clf.predict_proba(X_test)[:, 1]
            y_pred = clf.predict(X_test)
            cv_score = cross_val_score(clf, X_train, y_train, scoring='roc_auc', n_jobs=-1)
            X_test_accuracy = X_test
        else:
            y_pred_proba = clf.predict_proba(sig_test)[:, 1]
            y_pred = clf.predict(sig_test)
            cv_score = cross_val_score(clf, sig_train, y_train, scoring='roc_auc', n_jobs=-1)
            X_test_accuracy = sig_test

        
        accuracy.append(accuracy_score(y_test, y_pred))
        accuracy_test.append(clf.score(X_test_accuracy, y_test))
        

    accuracy_final = sum(accuracy) / len(accuracy)
    accuracy_test_fianl = sum(accuracy_test) / len(accuracy_test)
    

    
    file_name = f"{dataset}_{method}"
    if not method.startswith("ts"):
        file_name += f"_sig{sig_level}"
    if ts_scale:
        file_name += "_ts_scale"
    if standard_scale:
        file_name += "_standard_scale"
    if time_aug:
        file_name += "_time_aug"
    score_df = pd.DataFrame(columns=["Accuracy", "Accuracy_test"], index=["value"])
    score_df["Accuracy"] = accuracy
    score_df["Accuracy_test"] = accuracy_test_fianl
    score_df.to_csv(f"/content/drive/My Drive/results/{file_name}.csv")

    


In [ ]:
def sigkernel_auto(dataset, train_size = 0.8, ts_scale = True):
    X_train, y_train, X_test, y_test = write_alcoholic(dataset)

    accuracy = []
    accuracy_test = []
    accuracy_train = []

    svc_parameters = {'C': np.logspace(0, 4, 5), 'gamma': list(np.logspace(-4, 4, 9)) + ['auto']}
    _sigmas = [1e-2, 2.5e-2, 5e-2, 7.5e-2, 1e-1, 1.5e-1, 2e-1, 2.5e-1, 3e-1, 3.5e-1, 4e-1, 4.5e-1, 5e-1, 5.5e-1, 6e-1, 6.5e-1, 7e-1, 7.5e-1, 8e-1, 8.5e-1, 9e-1, 9.5e-1, 1.]
    _scales = [5e-2, 1e-1, 5e-1, 1e0]

    for i in range(5):
      X_train, X_test, y_train, y_test = shuffle_dataset(X_train, y_train, X_test, y_test, train_size)
      # initialise scalers
      ts_scaler = TimeSeriesScalerMinMax()

      if ts_scale:
          X_train = ts_scaler.fit_transform(X_train)
          X_test = ts_scaler.fit_transform(X_test)
        
      best_scores, trained_models = sigkernel_train(X_train, y_train, X_test, y_test)
      accuracy_sk, accuracy_test_sk, accuracy_train_sk = sigkernel_test(X_train, y_train, X_test, y_test, trained_models)
      accuracy.append(accuracy_sk)
      accuracy_test.append(accuracy_test_sk)
      accuracy_train.append(accuracy_train_sk)
      print(accuracy_test)
    

    accuracy_final = sum(accuracy) / len(accuracy)
    accuracy_test_fianl = sum(accuracy_test) / len(accuracy_test)
    accuracy_train_final = sum(accuracy_train) / len(accuracy_train)
    print(accuracy_final)
    print(accuracy_test_fianl)
    print(accuracy_train_final)
    

    file_name = f"{dataset}_sigkernel"
    if ts_scale:
        file_name += "_ts_scale"
    score_df = pd.DataFrame(columns=["Accuracy", "Accuracy_test", "Accuracy_train"], index=["value"])
    score_df["Accuracy"] = accuracy_final
    score_df["Accuracy_test"] = accuracy_test_fianl
    score_df["Accuracy_train"] = accuracy_train_final
    score_df.to_csv(f"/content/drive/My Drive/results/{file_name}.csv")

In [ ]:
def main(dataset, method, sig_level, ts_scale=True, standard_scale=True, time_aug=False, train_size=0.8):
    X_train, y_train, X_test, y_test = read_data(dataset)

    auto_ml(X_train, y_train, X_test, y_test, method, sig_level, dataset,
            ts_scale=ts_scale, standard_scale=standard_scale, time_aug=time_aug, train_size=train_size)
    

In [ ]:

for i in data_list[1:2]:
    for ts_scale in [True, False]:
      sigkernel_auto(i, train_size = 0.8, ts_scale = True)

            
        
                    
              

100%|██████████| 480/480 [00:17<00:00, 27.83it/s]

add-time: False, lead-lag: False:  50%|█████     | 1/2 [05:58<05:58, 358.60s/it]